In [19]:
import requests

app_id = 'bGKek0APxR3pVmIijdf_Gw'
app_secret = '0vobaKWAOvIZwAiYrI0pSg4PW6ZqRE6-injBpcu8NrSCML1_I6sUPKxir9j2a10wSxa6my_ouPXoNM52N6BT-dVqLJx3i8Im0b-8LIuHoPzJwSDtLpQTl64rU_vkW3Yx'
data = {'grant_type': 'client_credentials',
        'client_id': app_id,
        'client_secret': app_secret}
token = requests.post('https://api.yelp.com/oauth2/token', data=data)


In [20]:
token.json()

{u'error': {u'code': u'VALIDATION_ERROR',
  u'description': u"'client_credentials' is not one of ['authorization_code', 'refresh_token']",
  u'field': u'grant_type',
  u'instance': u'client_credentials'}}

In [ ]:
access_token = token.json()['access_token']
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % access_token}
params = {'location': 'San Bruno',
          'term': 'Japanese Restaurant',
          'pricing_filter': '1, 2',
          'sort_by': 'rating'
         }

resp = requests.get(url=url, params=params, headers=headers)

import pprint
pprint.pprint(resp.json()['businesses'])

In [21]:
from yelp.client import Client

In [22]:
MY_API_KEY = "0vobaKWAOvIZwAiYrI0pSg4PW6ZqRE6-injBpcu8NrSCML1_I6sUPKxir9j2a10wSxa6my_ouPXoNM52N6BT-dVqLJx3i8Im0b-8LIuHoPzJwSDtLpQTl64rU_vkW3Yx"

In [23]:
client = Client(MY_API_KEY)

In [24]:
client

In [27]:
client.get_business(business_id='yelp-san-francisco')

AttributeError: 'str' object has no attribute 'sign_request'

In [1]:

from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
import json_to_csv_converter
import pandas as pd
import csv


# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

In [5]:
API_KEY= '0vobaKWAOvIZwAiYrI0pSg4PW6ZqRE6-injBpcu8NrSCML1_I6sUPKxir9j2a10wSxa6my_ouPXoNM52N6BT-dVqLJx3i8Im0b-8LIuHoPzJwSDtLpQTl64rU_vkW3Yx'


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'late-night'
DEFAULT_LOCATION = 'New York City'
SEARCH_LIMIT = 100


def get_businesses1(location, term, api_key):
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = 'https://api.yelp.com/v3/businesses/search'

    data = []
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': location.replace(' ', '+'),
            'term': term.replace(' ', '+'),
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break

    return data

def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+')
        #'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
                        type=str, help='Search term (default: %(default)s)')
    parser.add_argument('-l', '--location', dest='location',
                        default=DEFAULT_LOCATION, type=str,
                        help='Search location (default: %(default)s)')

    input_values = parser.parse_args()

    try:
        query_api(input_values.term, input_values.location)
    except HTTPError as error:
        sys.exit(
            'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
                error.code,
                error.url,
                error.read(),
            )
        )

In [6]:
business = search(API_KEY, DEFAULT_TERM, DEFAULT_LOCATION)

Querying https://api.yelp.com/v3/businesses/search ...


In [7]:
business

{u'businesses': [{u'alias': u'levain-bakery-new-york',
   u'categories': [{u'alias': u'bakeries', u'title': u'Bakeries'}],
   u'coordinates': {u'latitude': 40.7799404643263,
    u'longitude': -73.980282552649},
   u'display_phone': u'(212) 874-6080',
   u'distance': 8367.197123060287,
   u'id': u'H4jJ7XB3CetIr1pg56CczQ',
   u'image_url': u'https://s3-media1.fl.yelpcdn.com/bphoto/qS--12qjadrrhQ--huKKuA/o.jpg',
   u'is_closed': False,
   u'location': {u'address1': u'167 W 74th St',
    u'address2': u'',
    u'address3': u'',
    u'city': u'New York',
    u'country': u'US',
    u'display_address': [u'167 W 74th St', u'New York, NY 10023'],
    u'state': u'NY',
    u'zip_code': u'10023'},
   u'name': u'Levain Bakery',
   u'phone': u'+12128746080',
   u'price': u'$$',
   u'rating': 4.5,
   u'review_count': 7060,
   u'transactions': [],
   u'url': u'https://www.yelp.com/biz/levain-bakery-new-york?adjust_creative=bGKek0APxR3pVmIijdf_Gw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_searc

In [46]:
len(business['businesses'])

20

In [32]:
data = get_businesses1(DEFAULT_LOCATION, DEFAULT_TERM, API_KEY)

In [33]:
len(data)

1000

In [50]:
data[0]['location']['zip_code']

u'10019'

In [40]:
x = str(data[0]['name'])
x

'The Halal Guys'

In [59]:
def to_string(s):
    try:
        return str(s)
    except:
        #Change the encoding type if needed
        return s.encode('utf-8')


##
# This function converts an item like 
# {
#   "item_1":"value_11",
#   "item_2":"value_12",
#   "item_3":"value_13",
#   "item_4":["sub_value_14", "sub_value_15"],
#   "item_5":{
#       "sub_item_1":"sub_item_value_11",
#       "sub_item_2":["sub_item_value_12", "sub_item_value_13"]
#   }
# }
# To
# {
#   "node_item_1":"value_11",
#   "node_item_2":"value_12",
#   "node_item_3":"value_13",
#   "node_item_4_0":"sub_value_14", 
#   "node_item_4_1":"sub_value_15",
#   "node_item_5_sub_item_1":"sub_item_value_11",
#   "node_item_5_sub_item_2_0":"sub_item_value_12",
#   "node_item_5_sub_item_2_0":"sub_item_value_13"
# }
##
def reduce_item(key, value):
    global reduced_item
    
    #Reduction Condition 1
    if type(value) is list:
        i=0
        for sub_item in value:
            reduce_item(key+'_'+to_string(i), sub_item)
            i=i+1

    #Reduction Condition 2
    elif type(value) is dict:
        sub_keys = value.keys()
        for sub_key in sub_keys:
            reduce_item(key+'_'+to_string(sub_key), value[sub_key])
    
    #Base Condition
    else:
        reduced_item[to_string(key)] = to_string(value)


In [53]:
with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

In [74]:
node = 'restaurant'
fp = open('data.json', 'r')
json_value = fp.read()
raw_data = json.loads(json_value)


#print(raw_data[])

try:
    data_to_be_processed = raw_data[node]
except:
    data_to_be_processed = raw_data

processed_data = []
header = []
for item in data_to_be_processed:
    reduced_item = {}
    reduce_item(node, item)

    header += reduced_item.keys()

    processed_data.append(reduced_item)

header = list(set(header))
header.sort()


In [75]:
with open('lateNightRest.csv', 'w+') as f:
    writer = csv.DictWriter(f, header, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for row in processed_data:
        writer.writerow(row)

print ("Just completed writing csv file with %d columns" % len(header))

Just completed writing csv file with 34 columns


In [36]:
y = get_business(API_KEY, x)

Querying https://api.yelp.com/v3/businesses/the-halal-guys-new-york-2 ...


In [37]:
y['hours']

[{u'hours_type': u'REGULAR',
  u'is_open_now': True,
  u'open': [{u'day': 0,
    u'end': u'0400',
    u'is_overnight': True,
    u'start': u'1000'},
   {u'day': 1, u'end': u'0400', u'is_overnight': True, u'start': u'1000'},
   {u'day': 2, u'end': u'0400', u'is_overnight': True, u'start': u'1000'},
   {u'day': 3, u'end': u'0400', u'is_overnight': True, u'start': u'1000'},
   {u'day': 4, u'end': u'0530', u'is_overnight': True, u'start': u'1000'},
   {u'day': 5, u'end': u'0530', u'is_overnight': True, u'start': u'1000'},
   {u'day': 6, u'end': u'0400', u'is_overnight': True, u'start': u'1000'}]}]

In [57]:
z = y['hours']
y['hours']

[{u'hours_type': u'REGULAR',
  u'is_open_now': False,
  u'open': [{u'day': 0,
    u'end': u'2130',
    u'is_overnight': False,
    u'start': u'1130'},
   {u'day': 1, u'end': u'2130', u'is_overnight': False, u'start': u'1130'},
   {u'day': 2, u'end': u'2130', u'is_overnight': False, u'start': u'1130'},
   {u'day': 3, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
   {u'day': 4, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
   {u'day': 5, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
   {u'day': 6, u'end': u'2130', u'is_overnight': False, u'start': u'1130'}]}]

In [59]:
z[0]['open']

[{u'day': 0, u'end': u'2130', u'is_overnight': False, u'start': u'1130'},
 {u'day': 1, u'end': u'2130', u'is_overnight': False, u'start': u'1130'},
 {u'day': 2, u'end': u'2130', u'is_overnight': False, u'start': u'1130'},
 {u'day': 3, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
 {u'day': 4, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
 {u'day': 5, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
 {u'day': 6, u'end': u'2130', u'is_overnight': False, u'start': u'1130'}]